In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spoc-dataset/spoc-train-test.tsv
/kaggle/input/spoc-dataset/spoc-train-eval.tsv
/kaggle/input/spoc-dataset/spoc-train-train.tsv


In [2]:
!pip install transformers datasets
!pip install rouge.score nltk py7zr

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.2/399.2 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.6/138.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 11.2 MB/s eta 0:00:00
  Created wheel for rouge.score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=9838589b92e0e7d89af98a6a1c221c8e54ee8c1465783733d673e2535707117d
  Stored in directory: /root/.cache/pip/wheels/8e/6b/70/59daa7c90a238610e34bac5916e001fe3d9bb0ec59c8cf5518
Successfully built rouge.score


In [3]:
training_sample = pd.read_table("/kaggle/input/spoc-dataset/spoc-train-train.tsv", usecols=["text","code"])
test_sample = pd.read_table("/kaggle/input/spoc-dataset/spoc-train-test.tsv", usecols=["text","code"])
eval_sample = pd.read_table("/kaggle/input/spoc-dataset/spoc-train-train.tsv", usecols=["text","code"])

In [4]:
%pip install rouge_score

Note: you may need to restart the kernel to use updated packages.


In [7]:
import transformers
from datasets import load_dataset, load_metric, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

metric = load_metric('rouge')
model_checkpoints = "facebook/bart-base"
#'facebook/bart-large-xsum'

[nltk_data] Downloading package punkt to /home/paxx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/tmp/ipykernel_280000/3904981882.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [6]:
max_input = 512
max_target = 128
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints,device_map="auto",load_in_8bit=True)

training_sample = training_sample.dropna()
test_sample = test_sample.dropna()
eval_sample = eval_sample.dropna()

training_sample = training_sample.reset_index(drop=True)
test_sample = test_sample.reset_index(drop=True)
eval_sample = eval_sample.reset_index(drop=True)

training_sample = training_sample.iloc[:100000]
test_sample = test_sample.iloc[:15000]
eval_sample = eval_sample.iloc[:15000]




In [7]:
def preprocess_data(dataset):

  inputs = [code for code in dataset['code']]
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  with tokenizer.as_target_tokenizer():
    targets = tokenizer(dataset['text'], max_length=max_target, padding='max_length', truncation=True)

  model_inputs['labels'] = targets['input_ids']
  return model_inputs

In [8]:
def preprocexss_data(dataset):
  inputs = [code for code in dataset['code']]
  model_inputs = tokenizer(inputs, max_length=max_input, padding='max_length', truncation=True)

  targets = tokenizer(dataset['text'], max_length=max_target, padding='max_length', truncation=True, text_pair=inputs, return_tensors='pt')

  model_inputs['labels'] = targets['input_ids']
  return model_inputs


In [9]:
from datasets import Dataset, load_dataset, DatasetDict
train = Dataset.from_dict(training_sample)
test = Dataset.from_dict(test_sample)
eval = Dataset.from_dict(eval_sample)

In [10]:
dataset = DatasetDict({"train" : train, "test": test,"eval": eval})
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'code'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['text', 'code'],
        num_rows: 15000
    })
    eval: Dataset({
        features: ['text', 'code'],
        num_rows: 15000
    })
})

In [11]:
dataset = dataset.map(preprocess_data, batched=True)
dataset

  0%|          | 0/100 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3587: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'code', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['text', 'code', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
    eval: Dataset({
        features: ['text', 'code', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
})

In [12]:
train_sample = dataset['train'].shuffle(seed=123)
validation_sample = dataset['eval'].shuffle(seed=123)
test_sample = dataset['test'].shuffle(seed=123)

dataset['train'] = train_sample
dataset['eval'] = validation_sample
dataset['test'] = test_sample


In [13]:

model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)
batch_size = 1
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
def compute_rouge(pred):
  predictions, labels = pred
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [15]:
args = transformers.Seq2SeqTrainingArguments(
    'code-pseudo',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )
trainer = transformers.Seq2SeqTrainer(
    model,
    args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['eval'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge,
    
)


In [16]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun May  7 08:33:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/

In [17]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
os.environ['WANDB_DISABLED'] = 'true'

print(os.environ['TOKENIZERS_PARALLELISM'])


# ce191d0118e9f898d13193aeb3466e8770e819d4

true


In [18]:
trainer.train()


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.038100,0.024668,82.842800,59.637800,81.332200,81.330700,10.961700
2,0.032100,0.020312,83.827900,61.309200,82.346800,82.346900,10.862300
3,0.026200,0.018195,84.490500,62.095400,82.936500,82.944100,10.830900


TrainOutput(global_step=150000, training_loss=0.04133844120025635, metrics={'train_runtime': 34738.6185, 'train_samples_per_second': 8.636, 'train_steps_per_second': 4.318, 'total_flos': 9.1460468736e+16, 'train_loss': 0.04133844120025635, 'epoch': 3.0})

In [19]:
save_path = 'BART_pretrained_model'
!mkdir {save_path}

In [20]:

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('BART_pretrained_model/tokenizer_config.json',
 'BART_pretrained_model/special_tokens_map.json',
 'BART_pretrained_model/vocab.json',
 'BART_pretrained_model/merges.txt',
 'BART_pretrained_model/added_tokens.json',
 'BART_pretrained_model/tokenizer.json')

In [21]:
code = ["""int main()"""
  """{
  int first_number, second_number, sum; ""","""
  cout << "Enter two integers: ";
  cin >> first_number >> second_number;""","""
  sum = first_number + second_number;""","""
  cout << first_number << " + " <<  second_number << " = " << sum;
  return 0;
}"""]

type(code[0])

codes_all =[]
type(codes_all)

def output_function(codes):
  codes_all =[]
  for code1 in codes:
    print(code1)
    model_inputs = tokenizer(code1,  max_length=max_input, padding='max_length', truncation=True)
    raw_pred, _, _ = trainer.predict([model_inputs])
    codes_all.append((tokenizer.decode(raw_pred[0])))

  print(codes_all)

output_function(code)


int main(){
  int first_number, second_number, sum; 



  cout << "Enter two integers: ";
  cin >> first_number >> second_number;



  sum = first_number + second_number;



  cout << first_number << " + " <<  second_number << " = " << sum;
  return 0;
}


['</s><s>declare main(){ occurring  integers first_number, second_number and sum</s><pad>', '</s><s>print "Enter two integers: "; Leban  cin, first_number,</s>', '</s><s>set sum to first_number + second_number</s><pad><pad><pad><pad><pad><pad><pad>', '</s><s>print first_number, " + " and  second_number and " = "</s>']


#### loading datasets

In [12]:
train_file_path = "Datasets_PART2/SPoC/train/split/spoc-train-train.tsv"
test_file_path = "Datasets_PART2/SPoC/train/split/spoc-train-test.tsv"
eval_file_path = "Datasets_PART2/SPoC/train/split/spoc-train-eval.tsv"
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(".")
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints,device_map="auto",load_in_8bit=True)

In [10]:
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [9]:
import pandas as pd

training_sample = pd.read_table(train_file_path, usecols=["text","code"])
test_sample = pd.read_table(test_file_path, usecols=["text","code"])
eval_sample = pd.read_table(eval_file_path, usecols=["text","code"])

training_sample = training_sample.dropna()
test_sample = test_sample.dropna()
eval_sample = eval_sample.dropna()

training_sample = training_sample.reset_index(drop=True)
test_sample = test_sample.reset_index(drop=True)
eval_sample = eval_sample.reset_index(drop=True)

bleu_sample = training_sample[100000:]
bleu_sample = bleu_sample[:1000]
bleu_test_set = bleu_sample['code']
bleu_refs = bleu_sample['text']
bleu_test_set = bleu_test_set.to_numpy()
bleu_refs = bleu_refs.to_numpy()

training_sample = training_sample.iloc[:100000]
test_sample = test_sample.iloc[:15000]
eval_sample = eval_sample.iloc[:15000]

#### evaluation metrics

In [13]:
from nltk.translate.bleu_score import corpus_bleu

test_set = bleu_test_set.tolist()

references = bleu_refs.tolist()

test_encodings = tokenizer(test_set, max_length=256, truncation=True, padding=True, return_tensors="pt")

predicted_ids = model.generate(test_encodings["input_ids"], attention_mask=test_encodings["attention_mask"])
predicted_texts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
print(predicted_texts)

bleu_score = corpus_bleu([[ref] for ref in references], predicted_texts)
print("BLEU score:", bleu_score)

/home/paxx/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023-05-08 03:44:17.518582: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 03:44:19.790175: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-08 03:44:19.790470: W tensorflow/compiler/xla/stream

['while decrement n is true', 'a = integer', 'read a', 'if a >= 0', 'increment sum by a', 'else', 'decrement sum by a', 'print sum', 'n = integer', 'read n', 'x = integer', 'b, c = int with b = 0 and c = 0', 'for i = 0 to n exclusive', 'read x', 'if x is less than 0', 'set b to b + x', 'else', 'set c to c + x', 'print c - b', 'in inline function read returning int', 'x, f = int with x = 0 and f = 1', 'create character ch with ch = read character', "while ch < '0' or ch > '9'", "if ch is '-', set f to -1", 'set ch to read character', "while ch >= '0' and ch <= '9'", "set x to 10 * x + ch - '0'", 'set ch to read character', 'return x * f', 'n = integer', 'set n to read()', 'create integer sum with sum = 0', 'while decrement n is true', 'create integer h with h = read()', 'if h is greater than 0', 'increment sum by h', 'else', 'decrement sum by h', 'print sum', 'n = integer', 'read n', 'a = integer', 'create integer sum with sum = 0', 'create integer cnt with cnt = 0', 'while decrement n 

In [15]:
from rouge import Rouge

test_set = bleu_test_set.tolist()

references = bleu_refs.tolist()

test_encodings = tokenizer(test_set, max_length=256, truncation=True, padding=True, return_tensors="pt")

predicted_ids = model.generate(test_encodings["input_ids"], attention_mask=test_encodings["attention_mask"])
predicted_texts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

rouge = Rouge()
rouge_scores = rouge.get_scores(predicted_texts, references, avg=True, ignore_empty=True)

print("ROUGE-1 score:", rouge_scores["rouge-1"]["f"])
print("ROUGE-2 score:", rouge_scores["rouge-2"]["f"])

ROUGE-1 score: 0.7152895021503278
ROUGE-2 score: 0.47882454041240885
